<a href="https://colab.research.google.com/github/Alfred9/Exploring-LLMs/blob/main/Visual%20Language%20Models/Vision_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install  Required Packages
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

!pip install git+https://github.com/huggingface/transformers.git@60bb571e993b7d73257fb64044726b569fef9403 pillow==10.3.0 chardet==5.2.0 flash_attn==2.5.8 accelerate==0.30.1 bitsandbytes==0.43.1


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Cloning https://github.com/huggingface/transformers.git (to revision 60bb571e993b7d73257fb64044726b569fef9403) to /tmp/pip-req-build-jyeq803g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jyeq803g
  Running command git rev-parse -q --verify 'sha^60bb571e993b7d73257fb64044726b569fef9403'
  Running command git fetch -q https://github.com/huggingface/transformers.git 60bb571e993b7d73257fb64044726b569fef9403
  Running command git checkout -q 60bb571e993b7d73257fb64044726b569fef9403
  Resolved https://github.com/huggingface/transformers.git to commit 60bb571e993b7d73257fb64044726b569fef9403
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor
from transformers import BitsAndBytesConfig
import torch

In [3]:
#Define Model ID
model_id ="Microsoft/Phi-3-vision-128k-instruct"

In [4]:
# Load processor
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# Define BitsAndBytes configuration for 4-bit quantization
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
#Load model with 4-bit quaninization an map to CUDA

model=AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    quantization_config=nf4_config,
    _attn_implementation='eager'

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Define initial chat message with image placeholder
messages = [{"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}]

# Download image from URL
url = "https://media.hswstatic.com/eyJidWNrZXQiOiJjb250ZW50Lmhzd3N0YXRpYy5jb20iLCJrZXkiOiJnaWZcL2JlY29taW5nLWRvY3Rvci5qcGciLCJlZGl0cyI6eyJyZXNpemUiOnsid2lkdGgiOjgyOH19fQ=="
image = Image.open(requests.get(url, stream=True).raw)

In [9]:
# Prepare prompt with image token
prompt = processor.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# Process prompt and image for model input
inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

# Generate text response using model
generate_ids = model.generate(
    **inputs,
    eos_token_id=processor.tokenizer.eos_token_id,
    max_new_tokens=500,
    do_sample=False,
)

# Remove input tokens from generated response
generate_ids = generate_ids[:, inputs["input_ids"].shape[1] :]

# Decode generated IDs to text
response = processor.batch_decode(
    generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

# Print the generated response
print(response)

The image depicts a group of medical professionals, likely doctors and nurses, gathered around a patient lying in a hospital bed. They appear to be discussing the patient's condition or treatment plan.
